## 위로봇 오복이 모델 프로세스

### Base Model Load
 - 출처 : https://github.com/snunlp/KR-SBERT

In [1]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('j5ng/et5-sentence-comfort')

/opt/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 데이터 불러오기

In [1]:
import pandas as pd
df = pd.read_excel("../resources/comfort_datasets.xlsx")

### 챗봇 테스트

In [3]:
query_embeddings = model.encode(
    df['user'].tolist(),
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=True
)

Batches: 100%|██████████████████████████████| 1078/1078 [05:24<00:00,  3.32it/s]


In [38]:
query = "자살할래"
query_embedding = model.encode(query, normalize_embeddings=True)

top_k = min(5, len(df))
cos_scores = util.pytorch_cos_sim(query_embedding, query_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"<입력 문장과 유사한 {top_k} 개의 문장>")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {df.loc[int(idx)]['user']} :{df.loc[int(idx)]['answer']} {'(유사도: {:.4f})'.format(score)}")

입력 문장: 자살할래
<입력 문장과 유사한 5 개의 문장>
1: 자살할까 :하루만 더 버티자고 마음먹고 일주일만 버티자고 마음먹고 한 달을 더 버티자 다짐하고 나면 모든 일이 잘 풀릴 거예요. 나쁜 생각하지 마시고 힘내세요 (유사도: 0.8999)
2: 자살할래. :선생님이 죽는다면 슬퍼할 사람이 너무 많을 거예요. 그렇다고 다른 사람들때문에 선생님이 살아야 하는 건 아니예요. 선생님 자신을 위해서 최선을 다해 살면 하루하루 소소한 일상이 행복을 가져다 줄 거예요. (유사도: 0.8653)
3: 자살하고싶어 :죽고 싶었던 하루였지만 그래도 살아줘서 고마워요. 오늘 하루도 고생 많았어요. 잘 견뎌내줬네요. 내일은 더 나은 삶이 기다리고 있을 거예요. 분명 작은 조그마한 변화가 있을 거예요. 토닥토닥 고생했고, 오늘 잠들 때는 좋은 꿈 행복한 꿈꿨으면 좋겠습니다 (유사도: 0.8273)
4: 자살하고싶어 :삶을 살다 보면 큰 행복은 아니어도 소소한 행복 같은 게 올 거예요. 그러니까 그런 생각 가지지 말고 살아주세요. 선생님 힘내세요. (유사도: 0.8273)
5: 자살하기로 했어 :잠시 한 번만 생각해봐요. 내가 죽으면 슬퍼할 사람들을요… 분명히 이 고통 또한 지나갈 거예요… 고통이 지나가기 전까지 제가 옆에서 지켜드릴게요. (유사도: 0.8236)


### Sbert 모델 ONNX 양자화(quantization) 

#### Onnx 모델로 변환

In [40]:
from transformers import T5EncoderModel, AutoTokenizer
from pathlib import Path

MODEL_PATH = "j5ng/et5-sentence-comfort"

T5EncoderModel._keys_to_ignore_on_load_unexpected = ["decoder.*"]
model = T5EncoderModel.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

from transformers import pipeline

encoder = pipeline(
    "feature-extraction",
    model=model,
    tokenizer=tokenizer,
    return_tensors=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [41]:
import transformers.convert_graph_to_onnx as onnx_convert
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_convert.convert_pytorch(encoder, opset=17, output=Path("encoder.onnx"),
	use_external_format=False)

Using framework PyTorch: 2.0.1
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Ensuring inputs are in correct order
head_mask is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask']
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



#### Onnx 모델 Uint8(0~255)로 가중치(Weight) 양자화

In [42]:
quantize_dynamic("encoder.onnx", "encoder.onnx_uint8.onnx", 
                 weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[/encoder/block.0/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/encoder/block.0/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/block.1/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/encoder/block.1/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/block.2/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/encoder/block.2/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/block.3/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/encoder/block.3/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/block.4/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/encoder/block.4/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/block.5/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/enco

### Onnx 모델로 "user"질문 임베딩

In [2]:
from onnxruntime import InferenceSession
from transformers import AutoTokenizer
import torch
import numpy as np
from tqdm import tqdm

MODEL_PATH = "j5ng/et5-sentence-comfort"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
sess = InferenceSession("../models/onnx/encoder.onnx_uint8.onnx" , providers=["CPUExecutionProvider"])

/opt/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def mean_pooling(model_output, attention_mask):
    model_output = torch.from_numpy(model_output[0])
    # First element of model_output contains all token embeddings
    token_embeddings = model_output
    attention_mask = torch.from_numpy(attention_mask)
    input_mask_expanded = attention_mask.unsqueeze(
        -1).expand(token_embeddings.size())
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask, input_mask_expanded, sum_mask

In [4]:
def embedding_query(query: str, normalize_embeddings=False) -> np.ndarray:
    # user turn sequence to query embedding
    model_inputs = tokenizer(query, return_tensors="pt")
    inputs_onnx = {k: v.cpu().detach().numpy()
                   for k, v in model_inputs.items()}
    sequence = sess.run(None, inputs_onnx)
    query_embedding = mean_pooling(
        sequence, inputs_onnx["attention_mask"])[0][0]

    if normalize_embeddings:
        query_embedding = query_embedding / \
            np.linalg.norm(query_embedding)

    return query_embedding.numpy()

In [5]:
onnx_embeddings = [ embedding_query(sen, normalize_embeddings=True) for sen in tqdm(df['user'].tolist())]

100%|█████████████████████████████████████| 34477/34477 [14:07<00:00, 40.67it/s]


In [6]:
np.save("onnx_embeddings.npy",onnx_embeddings)

### Faiss 벡터 양자화(PQ)

In [7]:
import faiss

In [8]:
embeddings = np.load("./onnx_embeddings.npy")

# IndexPQ 생성
d = embeddings.shape[1]
nbits = 8  # 각 부분벡터의 비트 수
m = 768  # 분할 수

# dot product 거리 측정을 사용하는 벡터 인코더
index = faiss.IndexPQ(d, m, nbits, faiss.METRIC_INNER_PRODUCT)  # PQ 색인 생성
index.train(embeddings)  # 색인 훈련
index.add(embeddings)  # 데이터 추가

In [9]:
# index 저장
faiss.write_index(index, "faiss_onnx_uint8")

### 챗봇 테스트

In [10]:
def reply(query: str):
    embedding = np.expand_dims(embedding_query(query, normalize_embeddings=True), axis=0)
    D, I = index.search(embedding, 5)
    return df.loc[I[0]]

In [78]:
reply("짝사랑 포기하는 방법").index.to_list()

[30948, 30935, 30927, 29123, 30947]

In [139]:
class RRF:
    def __init__(self):
        self.EMBED_WEIGHT: int = 0.6
        self.KEYWORD_WEIGHT: int = 0.4
    
    def get_ranking(self, query1_ids, query2_ids):
        # 중복 없는 모든 값들을 구합니다.
        all_values = list(set(query1_ids + query2_ids))

        # 결과를 저장할 딕셔너리를 초기화합니다.
        ranking = {}

        # 모든 값을 순회하면서 해당 값의 순위를 구합니다.
        for value in all_values:
            rank1 = result1.index(value) + 1 if value in result1 else None
            rank2 = result2.index(value) + 1 if value in result2 else None
            ranking[value] = [rank1, rank2]

        return ranking
    
    def reciprocal_rank(self, rank):
        """주어진 순위에 대한 Reciprocal Rank를 계산하는 함수"""
        try:
            return 1 / rank
        except TypeError:
            return 0.0
        
    def get_rrf_scores(self, query1_ids, query2_ids):
        
        ranking = get_ranking(query1_ids, query2_ids)
        
        ids = []
        scores = []

        for key in ranking.keys():
            # 각 검색 시스템의 순위
            embed_rank = ranking[key][0]
            keyword_rank = ranking[key][1]

            # 각 검색 시스템의 Reciprocal Rank 계산
            embed_rr = self.reciprocal_rank(embed_rank)
            keyword_rr = self.reciprocal_rank(keyword_rank)

            # 가중치가 적용된 Reciprocal Rank 계산
            rrf = (self.EMBED_WEIGHT * embed_rr) + (self.KEYWORD_WEIGHT * keyword_rr)

            scores.append(rrf)
            ids.append(key)

        sorted_scores = sorted(scores, reverse=True)
        sorted_ids = [key for _, key in sorted(zip(scores, ranking.keys()), reverse=True)]
        return {"ids": sorted_ids, "scores": sorted_scores}

In [143]:
result1 = [30948, 30935, 30927, 29123, 30947]
result2 = [30935, 30948, 30949, 30947, 29123]

In [144]:
rrf = RRF()

In [145]:
rrf.get_rrf_scores(result1, result2)

{'ids': [30948, 30935, 29123, 30947, 30927, 30949],
 'scores': [0.8, 0.7, 0.23, 0.22, 0.19999999999999998, 0.13333333333333333]}